In [64]:
import sys
import os.path as osp
import os
import subprocess
import json
sys.path.append('/home/grg/git/alfa')
os.chdir('/home/grg/git/alfa')

def shell_execute(cmd):
    proc = subprocess.Popen(cmd.split(' '), stdout=subprocess.PIPE, shell=False)
    (out, err) = proc.communicate()
    return out

## Simplifying brain-twisting endless commands
and minimize the chance of typos between two tries

Let's say I need to type these commands :
<pre>ANTS 3 -m CC[ /home/grg/data/ALFA_DWI/10013/T1/10013_mabonlm_nobias_spm_c2_d2.nii.gz, /home/grg/data/ALFA_DWI/10013/DWI/10013_B0_brain_seg_1.nii.gz, 1, 4] -r Gauss[0,3] -t Elast[1.5] -i 30x20x10 -o /home/grg/data/ALFA_DWI/10013/DWI/ants_elast_dwi-to-t1_.nii.gz

WarpImageMultiTransform 3 /home/grg/data/ALFA_DWI/10013/DWI/10013_MD.nii.gz /home/grg/data/ALFA_DWI/10013/DWI/10013_MD_t1space.nii.gz -R /home/grg/data/ALFA_DWI/10013/T1/10013_mabonlm_nobias.nii /home/grg/data/ALFA_DWI/10013/DWI/ants_elast_dwi-to-t1_Warp.nii.gz /home/grg/data/ALFA_DWI/10013/DWI/ants_elast_dwi-to-t1_Affine.txt
</pre> 

I'll first create a JSON giving a template of the commands.

In [50]:
j = {'ants_dwi_to_t1': u'ANTS 3 -m CC[ %s, %s, 1, 4] -r Gauss[0,3] -t Elast[1.5] -i 30x20x10 -o %s',
     'warp_md_to_t1': u'WarpImageMultiTransform 3 %s %s -R %s %s %s'}
json.dump(j, open('/tmp/templates.json','w'))

The $\%s$ represent parameters (such as filenames) that will be provided later.

This is like separating bones (<i>the command itself</i>) from the meat (<i>the data</i>).

$thesaurus$ will then fill in the ($\%s$) blanks. Just name the desired command and give the parameters in the same order as they appear in the command.

In [58]:
cmd = './thesaurus.py -n --name ants_dwi_to_t1 /tmp/templates.json target.nii moving.nii trans_'
print '$', cmd
print shell_execute(cmd)

cmd = './thesaurus.py -n --name warp_md_to_t1 /tmp/templates.json target.nii moving.nii output.nii trans_Warp.nii trans_Affine.txt'
print '\n$', cmd
print shell_execute(cmd)

$ ./thesaurus.py -n --name ants_dwi_to_t1 /tmp/templates.json target.nii moving.nii trans_
ANTS 3 -m CC[ target.nii, moving.nii, 1, 4] -r Gauss[0,3] -t Elast[1.5] -i 30x20x10 -o trans_


$ ./thesaurus.py -n --name warp_md_to_t1 /tmp/templates.json target.nii moving.nii output.nii trans_Warp.nii trans_Affine.txt
WarpImageMultiTransform 3 target.nii moving.nii -R output.nii trans_Warp.nii trans_Affine.txt



The $-n$ option is just a safety lock to display the command only and not to run it.

## Pushing it further..

$thesaurus$ can work with structured databases and preexisting filetypes.
Here let's define the various IO of the previous commands.

$@$ indicate input items that must exist prior to running the command.
$>$ indicate output items that will be generated by the command.

Naturally all the different filetypes have to be predefined and recognized by the database system.

In [65]:
j = {'ants_dwi_to_t1': [u'@ALFA Denoised Nobias SPM Dilated White matter',
         u'@ALFA DWI B0 Brain FSL FAST White matter',
         u'ALFA ANTS Elast DWI to T1 Transformation Template Filename'],
     'warp_md_to_t1': [u'@ALFA Mean Diffusivity Image',
         u'>ALFA DWI MD Map Warped to T1 space',
         u'@ALFA Denoised Nobias T1 Image',
         u'@ALFA ANTS Elast DWI to T1 Transformation',
         u'@ALFA ANTS Elast DWI to T1 Affine Transformation']}
json.dump(j, open('/tmp/templates_io.json', 'w'))

Then call the desired command by its keyname, with the desired subject and feel light...

In [68]:
import thesaurus as t
a = t.ALFAHelper() #jsonfile='/tmp/templates_io.json')
a.parse_command(subject='10013', name='warp_md_to_t1')

u'WarpImageMultiTransform 3 /home/grg/data/ALFA_DWI/10013/DWI/10013_MD.nii.gz /home/grg/data/ALFA_DWI/10013/DWI/10013_MD_t1space.nii.gz -R /home/grg/data/ALFA_DWI/10013/T1/10013_mabonlm_nobias.nii /home/grg/data/ALFA_DWI/10013/DWI/ants_elast_dwi-to-t1_Warp.nii.gz /home/grg/data/ALFA_DWI/10013/DWI/ants_elast_dwi-to-t1_Affine.txt'

Based on this file it can also recreate the sequence of a pipeline and that provide you with the progression state of a subject along this pipeline.

In [77]:
step1 = a.current_stage('10013')
print 'step:', step1, '\n'

step2 = a.current_stage('55855')
print 'step:', step2

10013 is complete
step: 0 

55855 is stuck at step before warp_aal_to_dwi (missing: ALFA ANTS Elast T1 to DWI Transformation)
step: warp_aal_to_dwi-1


OK now enough talking. Can you please run that d@£( step over my entire dataset ?

In [86]:
d = '/home/grg/data/ALFA_DWI/'
from glob import glob
subjects = [e.split('/')[-1] for e in glob(osp.join(d, '*')) if osp.isdir(e)]

In [89]:
a.generate_batch(subjects, step='ants_dwi_to_t1', batchfile='/tmp/batch.sh', create_workflow=True)

Failed 10779 /home/grg/data/ALFA_DWI/10779/DWI/10779_B0_brain_seg_1.nii.gz not found (type ALFA DWI B0 Brain FSL FAST White matter) while declared as input (or remove the leading @)
Failed snapshots /home/grg/data/ALFA_DWI/snapshots/T1/snapshots_mabonlm_nobias_spm_c2_d2.nii.gz not found (type ALFA Denoised Nobias SPM Dilated White matter) while declared as input (or remove the leading @)
Failed duplicates /home/grg/data/ALFA_DWI/duplicates/T1/duplicates_mabonlm_nobias_spm_c2_d2.nii.gz not found (type ALFA Denoised Nobias SPM Dilated White matter) while declared as input (or remove the leading @)


['21056',
 '11136',
 '12636',
 '10472',
 '10265',
 '77109',
 '11943',
 '10225',
 '10338',
 '10117',
 '10162',
 '12975',
 '10396',
 '11248',
 '55297',
 '13070',
 '11902',
 '21057c',
 '10822',
 '13244',
 '55809',
 '55278',
 '13242',
 '12067',
 '10682',
 '12810',
 '10330',
 '66293',
 '11620',
 '11415',
 '77175',
 '10025',
 '11291',
 '10461',
 '10151',
 '12878',
 '12308',
 '12861',
 '21092',
 '77130',
 '13215',
 '11461',
 '12704',
 '10051',
 '12904',
 '10518',
 '11137',
 '10312',
 '10385',
 '77151',
 '12812',
 '10090',
 '11666',
 '77195',
 '10703',
 '11152',
 '11641',
 '11592',
 '11550',
 '10841',
 '12324',
 '11191',
 '11133',
 '13164',
 '77056',
 '11071',
 '11201',
 '10645',
 '55708',
 '10108',
 '11219',
 '11416',
 '10016',
 '12122',
 '12330',
 '77034',
 '12140',
 '10552',
 '77217',
 '55855',
 '10482',
 '12125',
 '12841',
 '44491',
 '66048',
 '10692',
 '12079',
 '55469',
 '66029',
 '10881',
 '12015',
 '10550',
 '12417',
 '10988',
 '66368',
 '11047',
 '10367',
 '11778c',
 '11638',
 '10217'

In [92]:
b = open('/tmp/batch.sh').readlines()
for each in b[:20]:
    print each

ANTS 3 -m CC[ /home/grg/data/ALFA_DWI/21056/T1/21056_mabonlm_nobias_spm_c2_d2.nii.gz, /home/grg/data/ALFA_DWI/21056/DWI/21056_B0_brain_seg_1.nii.gz, 1, 4] -r Gauss[0,3] -t Elast[1.5] -i 30x20x10 -o /home/grg/data/ALFA_DWI/21056/DWI/ants_elast_dwi-to-t1_.nii.gz

ANTS 3 -m CC[ /home/grg/data/ALFA_DWI/11136/T1/11136_mabonlm_nobias_spm_c2_d2.nii.gz, /home/grg/data/ALFA_DWI/11136/DWI/11136_B0_brain_seg_1.nii.gz, 1, 4] -r Gauss[0,3] -t Elast[1.5] -i 30x20x10 -o /home/grg/data/ALFA_DWI/11136/DWI/ants_elast_dwi-to-t1_.nii.gz

ANTS 3 -m CC[ /home/grg/data/ALFA_DWI/12636/T1/12636_mabonlm_nobias_spm_c2_d2.nii.gz, /home/grg/data/ALFA_DWI/12636/DWI/12636_B0_brain_seg_1.nii.gz, 1, 4] -r Gauss[0,3] -t Elast[1.5] -i 30x20x10 -o /home/grg/data/ALFA_DWI/12636/DWI/ants_elast_dwi-to-t1_.nii.gz

ANTS 3 -m CC[ /home/grg/data/ALFA_DWI/10472/T1/10472_mabonlm_nobias_spm_c2_d2.nii.gz, /home/grg/data/ALFA_DWI/10472/DWI/10472_B0_brain_seg_1.nii.gz, 1, 4] -r Gauss[0,3] -t Elast[1.5] -i 30x20x10 -o /home/grg/data/A